In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import numpy as np
import pandas as pd
from functools import partial
from multiprocessing import Pool
import multiprocessing
import time
import re
import glob
import tqdm
### Extract sentiment and NER recognition (Drug, Symptom) and split them into three columns
# Create list of filenames
import glob
import pandas as pd
import tqdm   ## this is a ticker
data_dir = "/data1/covid_sentiment_decode/" 
files = sorted(glob.glob(data_dir+"*.csv"))
file = '|'.join(files) # Convert to string

# Process files in loop
import pandas as pd 
out_dir = '/data1/data_8t/ivermectin/sentiment/' # Location to save files
data_type = {'created_at':'string','id':'string','full_text':'string','user':'object',"extracted_entity_sentiment":'object',"extracted_entity":'object'}

for file in tqdm.tqdm(files):
    df = pd.read_csv(file, dtype=data_type, usecols=['created_at','id','full_text','user','extracted_entity_sentiment',"extracted_entity"],lineterminator='\n',keep_default_na=False)
    raw_len = len(df) # Total number of tweets in file
    print(raw_len)
    keys='Drug'

    df = df[df["extracted_entity_sentiment"].str.contains(fr"\b(?:{keys})\b",case=False)] # Filter data containing keys
    df = df.drop_duplicates(subset=['id']) # Remove duplicates
    print(len(df))
    print('-----------------------------------------')

    df.to_csv(out_dir+file.split("/")[-1],index=False)  # Save file
def Drug_Entity(entity): # Extract drug entities from entity extraction section
    drug_entity = []
    try:
        for i in eval(entity): 
            if i['type'] == 'Drug':
                drug_entity.append(i['value'].lower())
        return drug_entity
    except TypeError: # because some users has no text
        return '[]'
    
def Symp_Entity(entity): # Extract symptom entities from entity extraction section
    symp_entity = []
    try:
        for i in eval(entity): 
            if i['type'] == 'Symptom':
                symp_entity.append(i['value'].lower())
        return symp_entity
    except TypeError: # because some users has no text
        return '[]'

def Senti_Entity(entity): # Extract drug entities from entity extraction section
    senti_entity = []
    try:
        for i in eval(entity): 
            if i['type'] == 'Drug' and 'sentiment' in i:
                senti_entity.append(i['sentiment'].lower())
        return senti_entity
    except TypeError: # because some users has no text
        return '[]'

def parall_func(df):
    #df['keywords'] = df['full_text'].apply(getDrug)
    df['drug_entity'] = df['extracted_entity'].apply(Drug_Entity)
    df['sentiment']=df['extracted_entity_sentiment'].apply(Senti_Entity)
    return df


#pattern = re.compile('|'.join(symptom_keywords),flags=re.I)
# This function is used for multithreading, input is the dataframe you want to process and the corresponding function
def parallelize_dataframe(df, func, **kwargs):
    #CPUs = multiprocessing.cpu_count()
    num_partitions = 4 # number of partitions to split dataframe
    num_cores =  4 # number of cores on your machine

    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    func = partial(func, **kwargs)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

'''
# find the keywords in the text. if a keyword appears 
# several time in a text, only return one of it.
'''
import glob
import pandas as pd
import tqdm   ## this is a ticker
import seaborn as sns
import numpy as np
import pandas as pd
from functools import partial
from multiprocessing import Pool
import multiprocessing
import time
import re
import pandas as pd
import tqdm 
data_dir = "/data1/data_8t/ivermectin/sentiment/" 
out_dir='/data1/data_8t/ivermectin/sentiment/drug+senti/' # Folder to place results
files = sorted(glob.glob(data_dir+"*.csv"))
for file in tqdm.tqdm(files):
    df = pd.read_csv(file,lineterminator='\n',low_memory=False)
    df = parallelize_dataframe(df, parall_func)
    df.to_csv(out_dir+file.split("/")[-1],index=False)  # Save file